things to explain - backward induction for american option pricing

`numpy` things to know for the paper:
- `np.polyfit`
- `np.ploy1d`
- `np.polyval`
- `np.where`
- `np.random.standard_normal`
- `np.cumsum`

In [190]:
# imports
import numpy as np

In [191]:
temp = np.random.standard_normal((5,3))
print(temp)
print('')
print(np.cumsum(temp, axis=0))
print('')
print(np.cumsum(temp, axis=1))

[[ 0.32200914  1.32910831  2.38060024]
 [-0.56531126  0.19401598 -0.22280344]
 [ 0.13668262 -0.65860025  1.3377214 ]
 [ 1.87343121 -1.31038457 -0.05026927]
 [-0.14950003  0.69343764 -0.41378278]]

[[ 0.32200914  1.32910831  2.38060024]
 [-0.24330212  1.52312429  2.15779679]
 [-0.1066195   0.86452404  3.49551819]
 [ 1.76681171 -0.44586053  3.44524893]
 [ 1.61731167  0.24757711  3.03146615]]

[[ 0.32200914  1.65111745  4.03171769]
 [-0.56531126 -0.37129528 -0.59409872]
 [ 0.13668262 -0.52191763  0.81580378]
 [ 1.87343121  0.56304664  0.51277737]
 [-0.14950003  0.5439376   0.13015482]]


In the european option, only the terminal point is important.

In American options, the past is important, because the option can be exercised before the terminal point. Therefore we need to generate a matrix of random numbers and compute the cumulative sum because the stock price at time t is the cumulative sum of the previous stock returns.

In [192]:
# Longstaff & Schwartz algorithm
import math
import numpy as np
from math import exp, sqrt, log
np.random.seed(123456)

# input specifications
# S0 = 100.         # underlying price (S_0)
# false S0
S0=36.
# sigma = 0.20978   # volatility per year
# false sigma
sigma = 0.2
r = 0.06          # risk free rate per year
# div = 0.001       # dividend yield per year
# false div
div = 0.
mu = r - div      # expected returns under risk neutral measure
# T = 1.            # time to maturity
# false T:
T = 2.
# K = 110.          # exercise price
# false k
K=40


# Simulation parameters
I = 10**4                 # number of simulations
# M = 12                    # time steps (unit is years)
# false M
M = int(50/T)
dt = T/M                  # length of time steps
df = math.exp(-r * dt)    # discount factor

In [193]:
return_series = (mu - (sigma**2)/2.) * dt + sigma * sqrt(dt)* np.random.standard_normal((M, I))
S = S0 * np.exp(np.cumsum(return_series, axis = 0))
print(S.shape)
# note: shape is (M, I)
# print(S)

(25, 10000)


In [194]:
# compute inner value ( inner value == the value of the option if it is exercised right away )
h = np.maximum(K - S, 0.) # exercising right away
V = h[-1] # terminal value

#  Let us compute the value of american option with backward induction 
polyfit_order = 2
for t in range(M-1, 0, -1):
    rg = np.polyfit(S[t], V*df, polyfit_order) # estimate continuation-value function with stock prices
    C = np.polyval(rg, S[t]) # evaluate continuation value with stock prices with function
    V = np.where(h[t]>C, h[t], V*df) # update american value
    
V0 = df * np.average(V) # finally, compute the value of the american option 
print(f"The american option value is: {round(V0,4)}")

The american option value is: 4.6614


In [195]:
# 4.6875
# 4.667